In [6]:
%reload_ext dotenv
%dotenv
config_file_path='./config-dataunion-mumbai.ini'

In [7]:
import ocean_lib
import sys
import time
import os

print(f"Using python:{sys.version}")
print(f"Using ocean_lib: {ocean_lib.__version__}")

Using python:3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 16:22:27) 
[GCC 9.3.0]
Using ocean_lib: 0.8.5


In [8]:
from ocean_lib.config import Config
from ocean_lib.ocean.ocean import Ocean
config = Config(config_file_path)
ocean = Ocean(config)

print(f"config.network_url = '{config.network_url}'")
print(f"config.block_confirmations = {config.block_confirmations.value}")
print(f"config.metadata_cache_uri = '{config.metadata_cache_uri}'")
print(f"config.provider_url = '{config.provider_url}'")

# Create wallet
from ocean_lib.web3_internal.wallet import Wallet
dataunion_wallet = Wallet(
    ocean.web3,
    os.getenv('PRIVATE_KEY1'),
    config.block_confirmations,
    config.transaction_timeout,
)
print(f"alice_wallet.address = '{dataunion_wallet.address}'")

config.network_url = 'https://rpc-mumbai.maticvigil.com'
config.block_confirmations = 1
config.metadata_cache_uri = 'https://aquarius.dataunion.market'
config.provider_url = 'https://provider.dev.dataunion.market/'
alice_wallet.address = '0xdF1dEc52e602020E27B0644Ea0F584b6Eb5CE4eA'


In [ ]:
# Publish ALG datatoken
ALG_datatoken = ocean.create_data_token('ALG1', 'ALG1', dataunion_wallet, blob=ocean.config.metadata_cache_uri)
ALG_datatoken.mint(dataunion_wallet.address, to_wei(100), alice_wallet)
print(f"ALG_datatoken.address = '{ALG_datatoken.address}'")

# Specify metadata and service attributes, for "GPR" algorithm script.
# In same location as Branin test dataset. GPR = Gaussian Process Regression.
ALG_metadata =  {
    "main": {
        "type": "algorithm",
        "algorithm": {
            "language": "python",
            "format": "docker-image",
            "version": "0.1",
            "container": {
              "entrypoint": "python $ALGO",
              "image": "oceanprotocol/algo_dockers",
              "tag": "python-branin"
            }
        },
        "files": [
      {
        "url": "https://raw.githubusercontent.com/trentmc/branin/main/gpr.py",
        "index": 0,
        "contentType": "text/text",
      }
    ],
    "name": "gpr", "author": "Trent", "license": "CC0",
    "dateCreated": "2020-01-28T10:55:11Z"
    }
}
ALG_service_attributes = {
        "main": {
            "name": "ALG_dataAssetAccessServiceAgreement",
            "creator": alice_wallet.address,
            "timeout": 3600 * 24,
            "datePublished": "2020-01-28T10:55:11Z",
            "cost": 1.0, # <don't change, this is obsolete>
        }
    }

# Calc ALG service access descriptor. We use the same service provider as DATA
ALG_access_service = Service(
    service_endpoint=provider_url,
    service_type=ServiceTypes.CLOUD_COMPUTE,
    attributes=ALG_service_attributes
)

# Publish metadata and service info on-chain
ALG_ddo = ocean.assets.create(
  metadata=ALG_metadata, # {"main" : {"type" : "algorithm", ..}, ..}
  publisher_wallet=alice_wallet,
  services=[ALG_access_service],
  data_token_address=ALG_datatoken.address)
print(f"ALG did = '{ALG_ddo.did}'")